In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-price-prediction/train_with_satellite_embeddings.csv
/kaggle/input/house-price-prediction/test_with_satellite_embeddings.csv
/kaggle/input/house-price-prediction/house_images_test/1269.jpg
/kaggle/input/house-price-prediction/house_images_test/3863.jpg
/kaggle/input/house-price-prediction/house_images_test/623.jpg
/kaggle/input/house-price-prediction/house_images_test/2193.jpg
/kaggle/input/house-price-prediction/house_images_test/3750.jpg
/kaggle/input/house-price-prediction/house_images_test/2008.jpg
/kaggle/input/house-price-prediction/house_images_test/2081.jpg
/kaggle/input/house-price-prediction/house_images_test/3919.jpg
/kaggle/input/house-price-prediction/house_images_test/3757.jpg
/kaggle/input/house-price-prediction/house_images_test/4489.jpg
/kaggle/input/house-price-prediction/house_images_test/3138.jpg
/kaggle/input/house-price-prediction/house_images_test/3417.jpg
/kaggle/input/house-price-prediction/house_images_test/764.jpg
/kaggle/input/house-price

In [3]:
# Required for the CLIP architecture and weights loading
!pip install open-clip-torch 

# Standard dependencies for image processing and data handling
!pip install pandas numpy pillow tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00


In [16]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor, AutoModel
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import xgboost as xgb

In [48]:
import lightgbm as lgb
from sklearn.metrics import r2_score

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [17]:
# Configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
TRAIN_DIR = '/kaggle/input/house-price-prediction/house_images'
TRAIN_CSV = '/kaggle/input/house-price-prediction/train_with_satellite_embeddings.csv'
TEST_DIR = '/kaggle/input/house-price-prediction/house_images_test'
TEST_CSV = '/kaggle/input/house-price-prediction/test_with_satellite_embeddings.csv'

In [18]:
# Load DINOv2-Base
print(f"Using Device: {DEVICE}")
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base").to(DEVICE).eval()

Using Device: cuda


In [22]:
class FolderDataset(Dataset):
    def __init__(self, directory, processor, image_list):
        self.directory = directory
        self.processor = processor
        # Use the list provided from the CSV to ensure order
        self.file_names = image_list

    def __len__(self): return len(self.file_names)

    def __getitem__(self, idx):
        filename = str(self.file_names[idx])
        # Add .jpg extension if missing in CSV but present in folder
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            filename += '.jpg'
            
        path = os.path.join(self.directory, filename)
        try:
            image = Image.open(path).convert("RGB")
            pixel_values = self.processor(images=image, return_tensors="pt").pixel_values.squeeze(0)
            return pixel_values, filename
        except Exception:
            # Zero-vector fallback for broken/missing images
            return torch.zeros((3, 224, 224)), filename

def get_embeddings_ordered(df, img_dir):
    # Determine the ID column automatically (usually the first one)
    id_col = df.columns[0]
    print(f"Aligning embeddings with column: {id_col}")
    
    dataset = FolderDataset(img_dir, processor, df[id_col].tolist())
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    embs = []
    with torch.no_grad():
        for batch_pixels, _ in tqdm(loader, desc="Extracting Features"):
            outputs = model(batch_pixels.to(DEVICE)).pooler_output
            embs.append(outputs.cpu().numpy())
            
    # Convert to DataFrame
    emb_df = pd.DataFrame(np.vstack(embs), columns=[f"dino_{i}" for i in range(768)])
    
    # Concatenate side-by-side (Order is guaranteed because shuffle=False)
    return pd.concat([df.reset_index(drop=True), emb_df], axis=1)

# Usage:
# train_final = get_embeddings_ordered(train_tab, TRAIN_DIR)
# test_final = get_embeddings_ordered(test_tab, TEST_DIR)

In [23]:
train_final = get_embeddings_ordered(train_tab, TRAIN_DIR)

Aligning embeddings with column: price


Extracting Features: 100%|██████████| 507/507 [04:51<00:00,  1.74it/s]


In [25]:
train_final.shape

(16209, 1049)

In [27]:
test_tab = pd.read_csv(TEST_CSV)

In [28]:
test_final = get_embeddings_ordered(test_tab, TEST_DIR)

Aligning embeddings with column: id


Extracting Features: 100%|██████████| 169/169 [01:37<00:00,  1.73it/s]


In [29]:
train_final.to_csv('final_train.csv',index=False)
test_final.to_csv('final_test.csv',index=False)